In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
assert sys.version_info >= (3, 5)

from pathlib import Path
from pprint import pprint
import pandas as pd
import numpy as np

import tensorflow as tf
assert tf.__version__ >= "2.0"

fdir = Path.cwd()
print(fdir)
sys.path.append(str(fdir/'../src'))
from config import cfg

from train_nn import get_tfr_files
from tf_utils import (calc_records_in_tfr_folder, calc_examples_in_tfrecord,
                      _float_feature, _bytes_feature, _int64_feature)

from build_df import load_rna
from merge_meta_files import load_crossref, load_pdx_meta

from tfrecords import FEA_SPEC #, FEA_SPEC_NEW

GREEN = '\033[92m'
ENDC = '\033[0m'
def green(text):
    return GREEN + str(text) + ENDC

/vol/ml/apartin/projects/pdx-histo/nbs
Num GPUs Available:  4


In [2]:
label = '299px_302um'

# Get the dataframe with the metadata and RNA

In [3]:
rna = load_rna()
cref = load_crossref()
pdx = load_pdx_meta()

mrg_cols = ['model', 'patient_id', 'specimen_id', 'sample_id']

pdx = pdx.rename(columns={'tumor_site_from_data_src': 'csite_src',
                          'tumor_type_from_data_src': 'ctype_src',
                          'simplified_tumor_site': 'csite',
                          'simplified_tumor_type': 'ctype'})

# Add columns to rna by parsing the Sample col
patient_id = rna['Sample'].map(lambda x: x.split('~')[0])
specimen_id = rna['Sample'].map(lambda x: x.split('~')[1])
sample_id = rna['Sample'].map(lambda x: x.split('~')[2])
model = [a + '~' + b for a, b in zip(patient_id, specimen_id)]
rna.insert(loc=1, column='model', value=model, allow_duplicates=True)
rna.insert(loc=2, column='patient_id', value=patient_id, allow_duplicates=True)
rna.insert(loc=3, column='specimen_id', value=specimen_id, allow_duplicates=True)
rna.insert(loc=4, column='sample_id', value=sample_id, allow_duplicates=True)
rna = rna.sort_values(['model', 'patient_id', 'specimen_id', 'sample_id'])

print(rna.shape)
print(cref.shape)
print(pdx.shape)

(1727, 981)
(593, 8)
(97, 7)


In [4]:
# Remove bad samples with bad slides
cref = cref[~cref.image_id.isin(cfg.BAD_SLIDES)].reset_index(drop=True)
cref.shape

(587, 8)

### Does RNA-Seq contain duplicates?
It doesn't. Is that expected??

In [5]:
subset_cols = [c for c in rna.columns if c.startswith('ge_') ]
print('RNA duplicates', sum(rna.duplicated(subset=subset_cols, keep=False)))
rna.sort_values(['model', 'patient_id', 'specimen_id', 'sample_id'])[:2]

RNA duplicates 0


,Sample,model,patient_id,specimen_id,sample_id,ge_AARS,ge_ABCB6,ge_ABCC5,ge_ABCF1,ge_ABCF3,...,ge_ZMIZ1,ge_ZMYM2,ge_ZNF131,ge_ZNF274,ge_ZNF318,ge_ZNF395,ge_ZNF451,ge_ZNF586,ge_ZNF589,ge_ZW10
0,114348~004-R~ATHY22,114348~004-R,114348,004-R,ATHY22,10.704853,6.121954,8.135935,10.305598,9.342794,...,10.058650,9.862901,7.550604,8.112825,7.847540,9.301633,7.561569,8.552815,7.418640,8.48305
1,114348~004-R~ATHY22E99,114348~004-R,114348,004-R,ATHY22E99,10.186330,5.770623,7.534462,10.358337,9.236192,...,9.587639,9.661212,7.493153,7.752289,7.631402,9.034510,7.305307,7.932549,7.090391,8.61203


In [6]:
aa = rna.groupby('model').agg({'patient_id': 'nunique', 'specimen_id': 'nunique', 'sample_id': 'nunique'}).reset_index().sort_values('sample_id', ascending=False)
aa.reset_index(drop=True)

,model,patient_id,specimen_id,sample_id
0,562742~068-R,1,1,10
1,989133~093-R,1,1,9
2,521955~158-R6,1,1,9
3,466732~252-T,1,1,9
4,941728~121-R,1,1,8
...,...,...,...,...
322,155919~109-R,1,1,1
323,565248~004-R,1,1,1
324,422866~222-R5,1,1,1
325,731285~195-R,1,1,1


### For some samples, we have histology slides but not rna-seq
Specifically, we miss rna-seq for 37 samples.

In [7]:
# Note! for some samples, we have images but not the rna-seq

# Subset the columns
df1 = cref[mrg_cols + ['image_id']]
df2 = rna

# Merge meta files
mrg = df1.merge(df2, on=mrg_cols, how='inner').reset_index(drop=True)
print('cref', df1.shape)
print('rna ', df2.shape)
print('mrg ', mrg.shape)

cref (587, 5)
rna  (1727, 981)
mrg  (550, 982)


In [8]:
# Explore (merge and identify from which df the items are coming from)
# https://kanoki.org/2019/07/04/pandas-difference-between-two-dataframes/
# Find which items are missing in Yitan's file
mrg_outer = df1.merge(df2, on=mrg_cols, how='outer', indicator=True)
print('Outer merge', mrg_outer.shape)
print(mrg_outer['_merge'].value_counts())

miss = mrg_outer.loc[lambda x: x['_merge']=='left_only']
miss = miss.sort_values(mrg_cols, ascending=True)
print('\nMissing items', miss.shape)
display(miss.iloc[:3, :10])

Outer merge (1764, 983)
right_only    1177
both           550
left_only       37
Name: _merge, dtype: int64

Missing items (37, 983)


,model,patient_id,specimen_id,sample_id,image_id,Sample,ge_AARS,ge_ABCB6,ge_ABCC5,ge_ABCF1
6,114551~080-T,114551,080-T,M667M226,16056.0,NaN,NaN,NaN,NaN,NaN
10,114551~080-T,114551,080-T,M667M226C21,16127.0,NaN,NaN,NaN,NaN,NaN
7,114551~080-T,114551,080-T,M667M228,16058.0,NaN,NaN,NaN,NaN,NaN


In [9]:
# Consider filling the rnaseq from sample_ids of the sample model
rna[rna.model.isin(miss.model)][:2]

,Sample,model,patient_id,specimen_id,sample_id,ge_AARS,ge_ABCB6,ge_ABCC5,ge_ABCF1,ge_ABCF3,...,ge_ZMIZ1,ge_ZMYM2,ge_ZNF131,ge_ZNF274,ge_ZNF318,ge_ZNF395,ge_ZNF451,ge_ZNF586,ge_ZNF589,ge_ZW10
10,114551~080-T~M667M226C29,114551~080-T,114551,080-T,M667M226C29,11.622240,8.220932,7.306632,9.989162,9.186027,...,9.644486,7.913247,6.899494,7.100257,7.336669,9.721398,6.235037,7.321681,7.391286,8.801201
11,114551~080-T~M667M227C30,114551~080-T,114551,080-T,M667M227C30,11.572382,8.229940,8.719181,10.267242,9.719035,...,10.123617,9.616393,7.531453,7.859683,7.631402,10.103002,7.757533,8.994519,8.314485,9.170944


### Now merge 

In [9]:
# Merge cref and rna
cref_rna = cref[mrg_cols + ['image_id']].merge(rna, on=mrg_cols, how='inner').reset_index(drop=True)
print(cref_rna.shape)
# display(cref_rna[:2])

# Note that we also loose some samples when we merge with pdx metadata
data = pdx.merge(cref_rna, on=['patient_id', 'specimen_id'], how='inner').reset_index(drop=True)
print(data.shape)
# display(mrg[:2])

# Re-org cols
cols = ['Sample', 'model', 'patient_id', 'specimen_id', 'sample_id', 'image_id', 
        'csite_src', 'ctype_src', 'csite', 'ctype', 'stage_or_grade']
ge_cols = [c for c in data.columns if str(c).startswith('ge_')]
data = data[cols + ge_cols]
# display(data[:2])

(556, 982)
(548, 987)


# Update TFRecords

In [10]:
# Destination for the updated tfrecords
outpath = cfg.SF_TFR_DIR
outpath = Path(str(outpath) + '_updated')/label
os.makedirs(outpath, exist_ok=True)
outpath

PosixPath('/vol/ml/apartin/projects/slideflow-proj/PDX_FIXED_updated_updated/299px_302um')

In [11]:
# Create dict of slide ids. Each slide contain a dict with metadata.
assert sum(cref_rna.duplicated('image_id', keep=False)) == 0, 'There are duplicates of image_id in the df'

mt = {}  # dict to store all metadata
GE_TYPE = np.float32

# Note that we use cref_rna since the subequent merge with pdx further
# looses a few samples.
for i, row_data in cref_rna.iterrows():
    # Dict to contain metadata for the current slide
    slide_dct = {}

    # Meta cols
    meta_cols = [c for c in row_data.index if not c.startswith('ge_')]
    for c in meta_cols:
        slide_dct[c] = str(row_data[c])

    # RNA cols
    ge_cols = [c for c in row_data.index if c.startswith('ge_')]
    ge_data = list(row_data[ge_cols].values.astype(GE_TYPE))
    slide_dct['ge_data'] = ge_data
    
    slide = str(row_data['image_id'])
    mt[slide] = slide_dct
    
print(f'A total of {len(mt)} samples with image and rna data.')

A total of 556 samples with image and rna data.


In [12]:
# Obtain slide names for which we need to update the tfrecords
directory = cfg.SF_TFR_DIR/label
tfr_files = list(directory.glob('*.tfrec*'))

# Slide names from tfrecords
slides = [s.name.split('.tfrec')[0] for s in tfr_files]
print(f'A total of {len(slides)} original tfrecords.')

# Common slides (that have both image and rna data)
c_slides = [s for s in slides if s in mt.keys()]
print(f'A total of {len(c_slides)} samples that have tfrecords and rna data.')

print('Missing tfrecords for the following slides (bad quality of histology slides): ', sorted(set(mt.keys()).difference(set(c_slides))))
# print(sorted(cfg.BAD_SLIDES))

A total of 550 original tfrecords.
A total of 550 samples that have tfrecords and rna data.
Missing tfrecords for the following slides (bad quality of histology slides):  ['20729', '21836', '22232', '45983', '83742', '83743']


In [17]:
# Load tfrecords and update with new data
for i, s in enumerate(sorted(c_slides)):
    rel_tfr = str(s) + '.tfrecords'
    tfr = str(directory/rel_tfr)  #join(directory, rel_tfr)
    
    print(f"\r\033[K Updating {green(rel_tfr)} ({i+1} out of {len(c_slides)} tfrecords) ...", end="") 
    
    tfr_fname = str(outpath/rel_tfr)
    writer = tf.io.TFRecordWriter(tfr_fname)
    
    raw_dataset = tf.data.TFRecordDataset(tfr)
        
    for rec in raw_dataset:
        features = tf.io.parse_single_example(rec, features=FEA_SPEC)  # rec features from old tfrecord
        # tf.print(features.keys())

        # Extract slide name from old tfrecord and get the new metadata to be added to the new tfrecord
        slide = features['slide'].numpy().decode('utf-8')
        slide_meta = mt[slide]
        
        ex = tf.train.Example(features=tf.train.Features(
            feature={
                # old features
                'slide':       _bytes_feature(features['slide'].numpy()),  # image_id
                'image_raw':   _bytes_feature(features['image_raw'].numpy()),

                # new features
                'model':       _bytes_feature(bytes(slide_meta['model'], 'utf-8')),
                'patient_id':  _bytes_feature(bytes(slide_meta['patient_id'], 'utf-8')),
                'specimen_id': _bytes_feature(bytes(slide_meta['specimen_id'], 'utf-8')),
                'sample_id':   _bytes_feature(bytes(slide_meta['sample_id'], 'utf-8')),
                'image_id':    _bytes_feature(bytes(slide_meta['image_id'], 'utf-8')),
                'Sample':      _bytes_feature(bytes(slide_meta['Sample'], 'utf-8')),
                'ge_data':     _float_feature(slide_meta['ge_data']),
            }
        ))
        
        writer.write(ex.SerializeToString())
        
    writer.close()        

 Updating tfrecord 9970.tfrecords (550 out of 550 tfrecords) 9970.tfrecords .....

In [149]:
raw_dataset = tf.data.TFRecordDataset(tfr)
rec = next(raw_dataset.__iter__())
features = tf.io.parse_single_example(rec, features=FEA_SPEC)  # rec features from old tfrecord
tf.print(features.keys())

slide = features['slide'].numpy().decode('utf-8')
slide

['image_raw', 'slide']


'9970'

In [1]:
# Write single tfrecord
# ---------------------

# s = c_slides[0]
# rel_tfr = str(s) + '.tfrecords'
# tfr = str(directory/rel_tfr)

# raw_dataset = tf.data.TFRecordDataset(tfr)
# rec = next(raw_dataset.__iter__())
# features = tf.io.parse_single_example(rec, features=FEA_SPEC)  # rec features from old tfrecord
# tf.print(features.keys())

# # Extract slide name from old tfrecord and get the new metadata to be added to the new tfrecord
# slide = features['slide'].numpy().decode('utf-8')
# slide_meta = mt[slide]

# tfr_fname = str(outpath/rel_tfr)
# writer = tf.io.TFRecordWriter(tfr_fname)

# tf_ex = tf.train.Example(features=tf.train.Features(
#     feature={
#         # old features
#         'slide':       _bytes_feature(features['slide'].numpy()),     # image_id
#         'image_raw':   _bytes_feature(features['image_raw'].numpy()),
        
#         # new features
#         'model':       _bytes_feature(bytes(slide_meta['model'], 'utf-8')),
#         'patient_id':  _bytes_feature(bytes(slide_meta['patient_id'], 'utf-8')),
#         'specimen_id': _bytes_feature(bytes(slide_meta['specimen_id'], 'utf-8')),
#         'sample_id':   _bytes_feature(bytes(slide_meta['sample_id'], 'utf-8')),
#         'image_id':    _bytes_feature(bytes(slide_meta['image_id'], 'utf-8')),
#         'Sample':      _bytes_feature(bytes(slide_meta['Sample'], 'utf-8')),
#         'ge_data':     _float_feature(slide_meta['ge_data']),
#     }
# ))

# writer.write(tf_ex.SerializeToString())
# writer.close()

# Try to load a TFRecord

In [168]:
GE_LEN = len(slide_meta['ge_data'])

fea_spec_new = {
    'slide': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
    'image_raw': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=None),

    'model': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
    'patient_id': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
    'specimen_id': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
    'sample_id': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
    'image_id': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
    'Sample': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
    'ge_data': tf.io.FixedLenFeature(shape=(GE_LEN,), dtype=tf.float32, default_value=None),
}

In [ ]:
s = c_slides[0]
rel_tfr = str(s) + '.tfrecords'
tfr_path = str(outpath/rel_tfr)
raw_dataset = tf.data.TFRecordDataset(tfr_path)
rec = next(raw_dataset.__iter__())
features = tf.io.parse_single_example(rec, features=fea_spec_new)  # rec features from old tfrecord
tf.print(features.keys())